In [114]:
import pandas as pd

### Unindo os datasets
Unir as notas de bechdel com elenco e equipe com o merge:
- O primeiro parâmetro passado é dataframe da "esquerda" e o segundo, da direita. Os dois próximos parâmetros (`left_on` e `right_on`) dizem quais colunas serão consideradas pra fazer o merge. O `how='inner'` calcula a interseção entre essas duas colunas.

Primeiro é feito o merge do links_df com o bechdel_df, pois bechdel_df não possui o tmdbid diretamente, mas os outros sim.
obs.: o 'id' no `credits.csv` e no `movies_metadata.csv` é o tmdbid.

`reset_index(drop='True')` = reseta os indexes sem criar uma nova coluna para eles.

In [138]:
bechdel_df = pd.read_csv('datasets/Bechdel_detailed.csv')
links_df = pd.read_csv('datasets/movie-dataset/links.csv') # contém o id do imdb e tmdb de cada filme.
credits_df = pd.read_csv('datasets/movie-dataset/credits.csv')
imdb_df = pd.read_csv('datasets/movie-dataset/movies_metadata.csv')

# remove colunas não usadas.
bechdel_df = bechdel_df.drop(['Unnamed: 0', 'submitterid', 'date', 'visible', 'id'], axis=1)

# renomeia rating para não coincidir com o dos outros datasets.
bechdel_df = bechdel_df.rename(columns={'rating': 'bt_score'})

# junta os 3 dataframes e descarta colunas duplicadas.
bechdel_df = pd.merge(bechdel_df, links_df, left_on='imdbid', right_on='imdbId', how='inner').drop('imdbId', axis=1)
bechdel_df = pd.merge(bechdel_df, credits_df, left_on='tmdbId', right_on='id', how='inner').drop(['id', 'movieId'], axis=1)

# descarta itens sem cast e/ou crew. Reseta os index.
bechdel_df = bechdel_df[
	(bechdel_df.cast != '[]') & (bechdel_df.crew != '[]')
].reset_index(drop='True')

# UNINDO COM O MOVIES_METADATA.CSV

# transforma o dtype de todos os elementos da coluna 'id' em float pra poder fazer o merge. Alguns possuem strings que são então convertidas em NaN.
imdb_df['id'] = pd.to_numeric(imdb_df['id'], errors='coerce')

# transforma orçamento em float
imdb_df.budget = pd.to_numeric(imdb_df.budget, errors='coerce')

# remove colunas não usadas.
imdb_df = imdb_df.drop(['adult', 'belongs_to_collection', 'homepage', 'original_title', 'overview', 'poster_path', 'release_date', 'spoken_languages', 'status', 'tagline', 'video', 'title'], axis=1)

# faz o df final.
bechdel_df = pd.merge(bechdel_df, imdb_df, left_on='tmdbId', right_on='id', how='inner').drop('id', axis=1)

bechdel_df

/var/folders/x5/46svw_qj6hs7h_f0d9_k70rc0000gn/T/ipykernel_8001/296107520.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  imdb_df = pd.read_csv('datasets/movie-dataset/movies_metadata.csv')


,title,year,bt_score,dubious,imdbid,tmdbId,cast,crew,budget,genres,imdb_id,original_language,popularity,production_companies,production_countries,revenue,runtime,vote_average,vote_count
0,Sallie Gardner at a Gallop,1878.0,0.0,0.0,2221420.0,194079.0,"[{'cast_id': 5, 'character': 'Herself - Sallie...","[{'credit_id': '52fe4ce29251416c91103327', 'de...",0.0,"[{'id': 99, 'name': 'Documentary'}]",tt2221420,en,0.327841,"[{'name': 'Palo Alto Stock Farm', 'id': 84748}]","[{'iso_3166_1': 'US', 'name': 'United States o...",0.0,1.0,6.2,25.0
1,Accordion Player,1888.0,0.0,0.0,1758563.0,96882.0,"[{'cast_id': 1, 'character': 'Himself', 'credi...","[{'credit_id': '52fe49c39251416c750d2a8b', 'de...",0.0,"[{'id': 99, 'name': 'Documentary'}]",tt1758563,xx,0.212768,"[{'name': 'Whitley Partners', 'id': 48026}]","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]",0.0,1.0,4.4,18.0
2,"Monkeyshines, No. 1",1890.0,0.0,0.0,361921.0,32571.0,"[{'cast_id': 10, 'character': 'Himself', 'cred...","[{'credit_id': '52fe44d99251416c9101ef9f', 'de...",0.0,"[{'id': 35, 'name': 'Comedy'}]",tt0361921,en,1.163672,"[{'name': 'Edison Manufacturing Company', 'id'...","[{'iso_3166_1': 'US', 'name': 'United States o...",0.0,1.0,4.3,15.0
3,"Monkeyshines, No. 2",1890.0,0.0,0.0,416046.0,33315.0,"[{'cast_id': 1, 'character': 'Himself', 'credi...","[{'credit_id': '52fe450c9251416c91025b0b', 'de...",0.0,[],tt0416046,xx,0.399258,"[{'name': 'Edison Manufacturing Company', 'id'...","[{'iso_3166_1': 'US', 'name': 'United States o...",0.0,1.0,4.5,12.0
4,"Monkeyshines, No. 3",1890.0,0.0,0.0,416047.0,33317.0,"[{'cast_id': 1, 'character': 'Himself', 'credi...","[{'credit_id': '52fe450c9251416c91025b2f', 'de...",0.0,[],tt0416047,xx,0.276161,"[{'name': 'Edison Manufacturing Company', 'id'...","[{'iso_3166_1': 'US', 'name': 'United States o...",0.0,1.0,3.3,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7606,12 feet deep,2017.0,3.0,0.0,5143226.0,459928.0,"[{'cast_id': 0, 'character': 'McGradey', 'cred...","[{'credit_id': '592ec68d92514130da00d716', 'de...",0.0,"[{'id': 53, 'name': 'Thriller'}]",tt5143226,en,4.479536,"[{'name': 'Citizen Skull Productions', 'id': 9...","[{'iso_3166_1': 'US', 'name': 'United States o...",0.0,85.0,5.1,62.0
7607,Landline,2017.0,3.0,0.0,5737862.0,419459.0,"[{'cast_id': 0, 'character': 'Dana Jacobs', 'c...","[{'credit_id': '5863a7fcc3a3680ab600716c', 'de...",0.0,"[{'id': 35, 'name': 'Comedy'}]",tt5737862,en,1.811506,"[{'name': 'Magnolia Pictures', 'id': 1030}]","[{'iso_3166_1': 'US', 'name': 'United States o...",0.0,93.0,5.1,9.0
7608,Patient Zero,2018.0,1.0,0.0,3458254.0,295011.0,"[{'cast_id': 1, 'character': 'Dr. Gina Rose', ...","[{'credit_id': '55c8fd47c3a36849b40002ec', 'de...",0.0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",tt3458254,en,1.083454,"[{'name': 'Screen Gems', 'id': 3287}, {'name':...","[{'iso_3166_1': 'US', 'name': 'United States o...",0.0,0.0,10.0,2.0
7609,Iron Sky: The Coming Race,2019.0,3.0,0.0,3038708.0,302349.0,"[{'cast_id': 1, 'character': 'Wolfgang Kortzfl...","[{'credit_id': '5461f666c3a3686f4c001f27', 'de...",18000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",tt3038708,en,1.917649,"[{'name': '27 Films Production', 'id': 9282}, ...","[{'iso_3166_1': 'BE', 'name': 'Belgium'}, {'is...",0.0,0.0,0.0,0.0


In [158]:
total = len(bechdel_df) * 0.01
print('Valores nulos:')
print('Orçamento: %.2f%%' % (len(bechdel_df[bechdel_df.budget == 0])/total))
print('Bilheteria: %.2f%%' % (len(bechdel_df[bechdel_df.revenue == 0])/total))
print('Duração: %.2f%%' % (len(bechdel_df[bechdel_df.runtime == 0])))
print('Número de votos: %.2f%%' % (len(bechdel_df[bechdel_df.vote_count == 0])))
print('Nota média: %.2f%%' % (len(bechdel_df[bechdel_df.vote_average == 0])))
print('Língua: %.2f%%' % (len(bechdel_df[bechdel_df.original_language == 'xx'])/total))
print('Gênero: %.2f%%' % (len(bechdel_df[bechdel_df.genres == '[]'])/total))
print('Produtoras: %.2f%%' % (len(bechdel_df[bechdel_df.production_companies == '[]'])/total))
print('Países: %.2f%%' % (len(bechdel_df[bechdel_df.production_countries == '[]'])/total))

Valores nulos:
Orçamento: 46.91%
Bilheteria: 47.08%
Duração: 24.00%
Número de votos: 14.00%
Nota média: 22.00%
Língua: 0.11%
Gênero: 0.26%
Produtoras: 5.93%
Países: 2.01%
